# BERT를 이용한 Chatbot 구현

In [ ]:
# !pip install transformers

## Pretrained BERT의 [CLS] token을 이용한 Chatbot

* 사전 준비물
  1. Pretrain된 BERT 모델
    * BERT based multi-lingual model을 사용할 예정
  2. 질의응답 Dataset

* 진행 과정
  1. 사용자의 질문(qeury)를 입력받음
  2. query를 pretrained BERT의 입력으로 넣어, query 문장에 해당하는 [CLS] token hidden을 얻음
  3. 사전에 준비된 질의응답 Dataset에 존재하는 모든 질문들을 pretrained BERT의 입력으로 넣어, 질문들에 해당하는 [CLS] token hidden을 얻음
  4. query의 [CLS] token hidden과 질문들의 [CLS] token hidden간의 코사인 유사도를 구함
  5. 가장 높은 코사인 유사도를 가진 질문의 답변을 반환함
  6. 1~5 과정을 반복함

### Pretrained BERT Load
* 공개된 BERT base Multilingual 버전을 사용함

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.parameters

### 질의응답 Dataset 예시
* Question과 Answer 리스트에서 동일한 index가 서로 pair가 됨

In [ ]:
chatbot_Question = ['기차 타고 여행 가고 싶어','꿈이 이루어질까?','내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!'] # 질문
chatbot_Answer = ['꿈꾸던 여행이네요.','현실을 꿈처럼 만들어봐요.','더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!'] # 답변
print(chatbot_Question[:])
# ['기차 타고 여행 가고 싶어', '꿈이 이루어질까?', '내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!']
print(chatbot_Answer[:])
# ['꿈꾸던 여행이네요.', '현실을 꿈처럼 만들어봐요.', '더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!']

### [CLS] token을 얻기 위한 함수
* 입력으로 들어온 문장을 tokenizing함
  * 긴 문장은 truncation함
  * `add_special_tokens=True`를 하여 앞에 [CLS] 뒤에 [SEP] 부착
* model에 tokenizing 했던 결과값들을 input으로 넣음
  * `input_ids` : tokenizing된 문장의 각각의 vocab id들을 입력함
  * `attention_mask` : padding은 0, paddind이 아니면 1로 구분하여 관찰하도록 명시적으로 알려줌
  * `token_type_ids` : 각 token마다 해당 문장의 index를 알려줌(index는 문장이 입력된 순서를 따름, 문장이 1개인 경우 모든 token은 초기값 0을 가짐)

In [ ]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
        # model(**tokenized_sent) : tokenized_sent 안에 정보가 포함되어 있기 때문에 동일하게 동작함
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

* 768 차원의 vector값을 얻음

* query vector : CLS token vector

In [ ]:
query = '아 여행가고 싶다~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape) # (1, 768)

* Chatbot 데이터셋의 Question 정보를 [CLS] token으로 바꿈

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Question:
    q_cls = get_cls_token(q)
    dataset_cls_hidden.append(q_cls)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)



* [CLS] token hidden 확인

In [ ]:
print(dataset_cls_hidden)   # 데이터셋의 질문에 대한 [CLS] 토큰 벡터
'''
[[ 0.2015294   0.00450035  0.19752151 ...  0.54929185  0.02755525
   0.21813345]
 [-0.09731648  0.10722532 -0.23476414 ...  0.36494994  0.454968
   0.31987903]
 [-0.23625095  0.13388395 -0.28571615 ...  0.53231657  0.33486852
   0.30000678]
 [ 0.02712039  0.12172284  0.07334824 ...  0.23747024  0.2893383
  -0.30687687]
 [-0.2990097  -0.0900768   0.10507746 ...  0.3788135   0.38025463
   0.30487213]]
'''

print(dataset_cls_hidden.shape) # (5, 768)

### 코사인 유사도
* 얻어낸 Question에 대한 각각의 vector들과 내가 원하는 query와 그 둘 간의 cosin similarlity를 비교함

In [ ]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)   # 데이터셋의 0번째 질문과 가장 유사하군요!
print(cos_sim)
# [[0.85016316 0.7788856  0.73615134 0.77987427 0.7242017 ]]

* Chatbot 데이터셋 중 가장 유사도가 높은 질문 선택 및 답변

In [ ]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
# 나의 질문:  아 여행가고 싶다~
print('저장된 답변: ', chatbot_Answer[top_question])
# 저장된 답변:  꿈꾸던 여행이네요.